In [1]:
%load_ext autoreload
%autoreload 2
from llatb import GameData, Live, DefaultLive, TeamBuilder, html_view, update_data
from llatb.skill import CenterSkill
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

#### Step 0: 使用前请先更新卡牌信息和 Live 信息

In [2]:
update_data()

Generating basic card stats
Basic card data has been saved in assets/data_base.json
Generating basic live stats
Basic live data has been saved in assets/live_data_base.json


#### Step1: 从 tshark json 抓包或者 ieb 数据站导入用户信息（注意抓包文件包含账户信息请不要随意给别人）

In [3]:
# user_profile = GameData('packet.txt', file_type='packet') # Load from tshark json packet
user_profile = GameData('ieb_sample.json', file_type='ieb') # Load from ieb format

Invalid team: 4-th team only has 5 members placed
Invalid team: 5-th team only has 5 members placed
Invalid team: 9-th team only has 8 members placed


#### Step2: 选择想要优化卡组的谱面

In [4]:
from llatb import show_live_list
group, attr, difficulty = "μ's", 'Cool', 'Master'
group, attr, difficulty = "Aqours", 'Smile', 'Expert'
show_live_list(group, attr, difficulty)

cover,name,diff_level,note_number,diff_star
,決めたよHand in Hand,Expert,569,10
,Aqours☆HEROES,Expert,515,10
,届かない星だとしても,Expert,563,10
,元気全開DAY!DAY!DAY!,Expert,592,10
,青空Jumping Heart,Expert,471,10
,Pops heartで踊るんだもん！,Expert,532,10
,MIRAI TICKET,Expert,536,10
,"Guilty Night, Guilty Kiss!",Expert,443,9
,HAPPY PARTY TRAIN,Expert,498,10
,太陽を追いかけろ！,Expert,476,9


可以使用默认Live设定, 也可以选择一个存在的谱面

In [5]:
# song_name, difficulty = "Default μ's Smile", 'Master'
# song_name, difficulty = "Default Aqours Cool", 'Expert'
# live = DefaultLive(song_name, difficulty)

song_name, difficulty = '太陽を追いかけろ！', 'Expert'
live = Live(song_name, difficulty)
html_view(live)

#### Step3:  给定应援加成信息，构造 Team Builder 并生成最优卡组

In [6]:
guest_cskill = CenterSkill(name='blah', main_attr='Cool', base_attr='Smile', 
						   main_ratio=12, bonus_range='BiBi', bonus_ratio=6)
opt = {'score_up_bonus':0.1, 'skill_up_bonus':0.1, 'guest_cskill':guest_cskill}
# opt = {'score_up_bonus':0, 'skill_up_bonus':0, 'guest_cskill':None}
tb = TeamBuilder(live, user_profile, opt=opt)
tb.build_team(K=15, method='4-suboptimal', alloc_method='DP')

Team searching method: 4-suboptimal. Gem allocation searching method: DP
1/5: Best team has score 662416 for Smile+Cool12%/BiBi6%
2/5: Best team has score 682886 for Smile+Pure12%/Printemps6%
3/5: Best team has score 679963 for Smile+Smile9%/μ's3%
4/5: Best team has score 677262 for Smile+Smile9%/1st-year6%
5/5: Best team has score 673100 for Smile+Pure12%/1st-year6%


如果使用的是存在的谱面，可以进行仿真查看模拟结果，还可以将一次仿真的结果输出成html文件观察

In [7]:
# tb.simulate() # Do not save similation detail to file
tb.simulate(save_to='test.html')

File saved to test.html


将最优卡组导出成为 LL Helper 格式，或 ieb 数据站格式

In [8]:
tb.best_team.to_LLHelper('team.sd')
tb.best_team.to_ieb('team.ieb')

File saved to team.sd
File saved to team.ieb


## 其他功能
可以将用户信息导出为 LL组卡器 支持的格式，使用 LL组卡器 计算最优卡组

In [9]:
user_profile.to_LLTB('cards.666')

file saved to cards.666


查看卡牌信息，排序按照入手时间

In [10]:
html_view(user_profile.raw_card)

,CID,Card View,Skill & Center Skill,,,,,,,skill_gain
,,,,,,,,,,
1,600,,にこちゃんと遊ぼう！ lv1 71% chance to raise the accuracy judge for 7.0 seconds Triggers every 25 seconds. Raise the team's Cool by 9% Raise the Cool of μ's members by 3%.,100 100,1000 1000,5,4570,4030,5320,14.6586% covered
2,720,,トータルチャーム lv6 46% chance to add 345.0 score points Triggers every 20 notes. Raise the team's Smile by 3%,100 100,1000 1000,5,4240,3400,2830,7.9350 pt/note
3,740,,雪玉合戦！ lv2 35% chance to raise the accuracy judge for 3.5 seconds Triggers every 18 notes. Raise the team's Pure by 6%,80 80,500 500,4,3160,4960,4220,37.0506% covered
4,746,,血を吸っちゃうぞ lv2 44% chance to raise the accuracy judge for 3.5 seconds Triggers every 26 notes. Raise the team's Cool by 6%,80 80,500 500,4,3470,3890,4820,33.3667% covered
5,760,,あったかいもの lv2 30% chance to restore 3.0 stamina points Triggers every 19 notes. Raise the team's Pure by 6%,80 80,500 500,4,4340,4980,3060,0.0463 hp/note
6,747,,パーフェクトチャーム lv8 50% chance to add 400.0 score points Triggers every 15 perfect notes. Raise the team's Smile by 3%,100 100,1000 1000,5,4230,3890,3690,12.5238 pt/note
7,744,,秘密のプレゼント lv3 38% chance to add 1285.0 score points Triggers every 21 combo. Raise the team's Smile by 12% of its Cool Raise the Smile of BiBi members by 6%.,80 80,500 500,4,5040,3780,4180,23.0199 pt/note
8,799,,妹にしてあげる♪ lv1 29% chance to add 600.0 score points Triggers every 19 notes. Raise the team's Pure by 12% of its Smile Raise the Pure of BiBi members by 6%.,100 100,1000 1000,5,4490,5320,4080,8.9486 pt/note
9,812,,妖精さんの願い事 lv2 22% chance to restore 4.0 stamina points Triggers every 20 notes. Raise the team's Cool by 6%,80 80,500 500,4,4050,3340,4990,0.0440 hp/note


查看卡组信息

In [11]:
html_view(user_profile.team_list[0], show_gem=True)

自定义搜索排序，如将所有红色非特典UR按照(Cool-Pure)的差值降序排列

In [12]:
sel_func = lambda x: x.main_attr=='Smile' and x.promo==False and x.rarity=='UR'
sort_func = lambda x: x.cool-x.pure
df = user_profile.filter(sel_func=sel_func, sort_func=sort_func, ascend=False)
html_view(df, extra_col=['sort_val'])

,CID,Card View,Skill & Center Skill,,,,,,,skill_gain,sort_val
,,,,,,,,,,,
80,1105,,ねずみ担当だから lv1 18% chance to restore 6.0 stamina points Triggers every 19 notes. Raise the team's Smile by 9% Raise the Smile of 1st-year members by 6%.,80 80,500 500,5,5070,3610,4370,0.0555 hp/note,760
7,744,,秘密のプレゼント lv3 38% chance to add 1285.0 score points Triggers every 21 combo. Raise the team's Smile by 12% of its Cool Raise the Smile of BiBi members by 6%.,80 80,500 500,4,5040,3780,4180,23.0199 pt/note,400
65,507,,恋をした天女 lv1 23% chance to raise the accuracy judge for 5.0 seconds Triggers every 17 notes. Raise the team's Smile by 9% Raise the Smile of μ's members by 3%.,80 80,500 500,4,5000,3840,4120,33.7381% covered,280
31,105,,伝説のラーメン！ lv1 36% chance to restore 4.0 stamina points Triggers every 27 notes. Raise the team's Smile by 9% Raise the Smile of μ's members by 3%.,80 80,500 500,4,4990,4040,3860,0.0514 hp/note,-180
12,79,,笑顔のステージ lv1 36% chance to raise the accuracy judge for 5.5 seconds Triggers every 30 notes. Raise the team's Smile by 9% Raise the Smile of μ's members by 3%.,100 100,1000 1000,5,5220,4490,4080,37.0244% covered,-410
10,843,,今は何の時代？ lv1 24% chance to restore 5.0 stamina points Triggers every 21 notes. Raise the team's Smile by 12% of its Pure Raise the Smile of 1st-year members by 6%.,100 100,1000 1000,6,5340,4620,3990,0.0566 hp/note,-630
21,315,,小さなお姫様 lv4 47% chance to add 1545.0 score points Triggers every 22 perfect notes. Raise the team's Smile by 12% of its Pure Raise the Smile of Printemps members by 6%.,100 100,1000 1000,5,5210,4750,3830,30.8378 pt/note,-920
